<a href="https://colab.research.google.com/github/akutayaydin/Magnimind-6-Image-Processing/blob/main/TNT_Project_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image
from skimage import io, filters
import scipy.ndimage as ndi
import sys
import keras.utils as image
from tensorflow.keras.utils import load_img
from google.colab.patches import cv2_imshow

In [54]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
unlabeled_image = cv2.imread('/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05.png')
#unlabeled_image.dtype

In [ ]:
#Apply Filters

In [ ]:
#Rolling Ball Algorithm : to remove uneven backgrounds or smooth the intensity variations in an image
# where the image may have uneven lighting or background artifacts

In [ ]:
import cv2
import numpy as np

def background_and_shading_correction(image, ball_radius):
    # Convert the image to grayscale, rolling ball algorithm is usually applied to grayscale images.
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply the rolling ball algorithm for background correction
    #The kernel_size is set to be twice the ball_radius plus one.
    #The "+1" ensures that the kernel has an odd size, which is a requirement for morphological operations.
    kernel_size = ball_radius * 2 + 1
    background_corrected = cv2.morphologyEx(gray_image, cv2.MORPH_TOPHAT, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size)))

    # Perform shading correction by subtracting the background corrected image from the original image
    shading_corrected = cv2.subtract(gray_image, background_corrected)

    # Convert the shading corrected image back to color (optional)
    shading_corrected_color = cv2.cvtColor(shading_corrected, cv2.COLOR_GRAY2BGR)

    # Combine the shading-corrected image with the original image to get the final result
    result_image = cv2.addWeighted(image, 1, shading_corrected_color, 0.7, 0)

    return result_image

# Load your input image
input_image = cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05.png")

# Set the radius for the rolling ball algorithm (adjust as needed)
ball_radius = 50

# Perform background and shading correction
output_image = background_and_shading_correction(input_image, ball_radius)

# Save the processed image (replace "output_image.jpg" with the desired output path)
cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/output_image_rolling_ball_50.png", output_image)



True

In [ ]:
import cv2

def background_and_shading_correction(image, sigma):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply the Gaussian filter for background correction
    background_corrected = cv2.GaussianBlur(gray_image, (0, 0), sigmaX=sigma, sigmaY=sigma)

    # Perform shading correction by subtracting the background corrected image from the original image
    shading_corrected = cv2.subtract(gray_image, background_corrected)

    # Convert the shading corrected image back to color (optional)
    shading_corrected_color = cv2.cvtColor(shading_corrected, cv2.COLOR_GRAY2BGR)

    # Combine the shading-corrected image with the original image to get the final result
    result_image = cv2.addWeighted(image, 1, shading_corrected_color, 0.7, 0)

    return result_image


# Load input image
input_image = cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05.png")

# Set the sigma value for the Gaussian filter (adjust as needed)
sigma = 3

# Perform background and shading correction
output_image = background_and_shading_correction(input_image, sigma)

# Save the processed image (replace "output_image.jpg" with the desired output path)
cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/output_image_gaussian.png", output_image)


True

In [24]:
#Step 1:Read and Slice the Preprocessed Image
#Step 1.1: Slice the unlabel image in the steps of region size

In [100]:
img_preprocessed = cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/output_image_rolling_ball_50.png")

#Set the region size
region_size = 512

#Get the dimensions of the image
height, width, channels = img_preprocessed.shape

#Iterate through the image in the steps of region_size
for y in range(0,height,region_size):
  for x in range (0,width,region_size):
    #Extract the region
    region = img_preprocessed[y:y+region_size,x:x+region_size]

    #Label the image based on the coordinates
    image_name = 'IMG' + '_'+ str(y) + '_' + str(x)

    #Create the image
    cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/" + image_name + ".png", region)

In [ ]:
#Step 1.2: Slice the labeled image in the steps of region_size

In [101]:
img_labeled =  cv2.imread("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/m05-label.png")

#Set the region size
region_size =512

for y in range(0,height,region_size):
  for x in range(0,width,region_size):
    #Extract the region
    region = img_labeled[y:y+region_size,x:x+region_size]

    #label the image based on the coordinates
    image_name = 'IMG' + '_' + str(y) + '_' + str(x)

    #Create and save the image file
    cv2.imwrite("/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_labeled_images/" + image_name + ".png", region)


In [47]:
#Step 2: Identify if there is tnt tube (yellow pixel) on the labeled image and update file name

In [55]:
folder_path = '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_labeled_images/'
folder_path_unlabeled = '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/'

In [103]:
lower_bound = np.array([0,100,100])  #lower bound BGR values for color
upper_bound = np.array([50,255,255]) #upper bound BGR values for color

#list all file names and create a mask for the target color
for f in os.listdir(folder_path):
  img_labeled_file = cv2.imread(folder_path +f)
  tnt_mask = cv2.inRange(img_labeled_file,lower_bound,upper_bound)

  # Count the number of color pixels
  color_pixel_count = cv2.countNonZero(tnt_mask)

  if color_pixel_count <= 50 and '_tnt_0.png' not in f:
    #rename file name
    f_new = f.replace ('.png','_tnt_0.png')
    os.rename (folder_path + f,folder_path + f_new)
    os.rename (folder_path_unlabeled + f,folder_path_unlabeled + f_new)
    folder_path_unlabeled
  elif color_pixel_count > 50 and '_tnt_1.png' not in f:
    #rename file name
    f_new = f.replace ('.png','_tnt_1.png')
    os.rename (folder_path + f,folder_path + f_new)
    os.rename (folder_path_unlabeled + f,folder_path_unlabeled + f_new)

In [60]:
# Step 3: Build a model

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16

In [4]:
#Step 3.1 Load pre-trained VGG16 model

In [35]:
base_model = keras.applications.vgg16.VGG16()

In [36]:
# Step 3.2 Create a sequential model
tnt_model = Sequential()

In [37]:
# Step 3.3 Remove prediction layer
pred_layer = base_model.layers[-1]

for layer in base_model.layers:
  try:
    if layer != pred_layer:
      tnt_model.add(layer)
  except TypeError as e:
    print(e)

# Step 3.4 Freeze layers to cut down on computational time for training
for layer in tnt_model.layers:
  layer.trainable = False

In [38]:
  # Step 3.5 Add a new output layer
  tnt_model.add(Dense(1,activation='sigmoid'))

In [41]:
# Step 3.6 Compile the network with an adam optimizer and binary cross-entropy loss and compute the accuracy during training
tnt_model.compile(optimizer = 'adam', loss = 'binary_crossentrophy', metrics = 'accuracy')

In [43]:
from keras.preprocessing.image import ImageDataGenerator

In [79]:
directory_name_train = '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/train'
directory_name_validation = '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/validation'

if not os.path.exists('directory_name_train'):
    # Create the directory
    os.makedirs('directory_name_train')
    print(f"Directory '{directory_name_train}' created successfully.")
else:
    print(f"Directory '{directory_name_train}' already exists.")


if not os.path.exists('directory_name_validation'):
    # Create the directory
    os.makedirs('directory_name_validation')
    print(f"Directory '{directory_name_validation}' created successfully.")
else:
    print(f"Directory '{directory_name_validation}' already exists.")

Directory '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/train' already exists.
Directory '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images/validation' created successfully.


In [98]:
file_path = '/content/gdrive/My Drive/Google Colab Folder/TNT_Project/sliced_unlabeled_images'

def get_class_label(filename):
  parts = filename.split('_') # Split the file name by underscores
  class_label = parts[-1].split('.')[0] # Remove the file extension
  return class_label

# Create an instance of ImageDataGenerator
traindatagen = ImageDataGenerator(rescale = 1./255,
                                         shear_range = 0.2,
                                         zoom_range = 0.2,
                                         horizontal_flip = True,
                                         rotation_range = 40)

# Load and preprocess the training dataset
train_generator = traindatagen.flow_from_directory(
    file_path,
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'sparse',
    #class_mode = None
    labels = [get_class_label(file) for file in train_generator.filenames]
)

labels = [get_class_label(file) for file in train_generator.filenames]
train_generator.classes = labels


TypeError: ignored

In [97]:
filepath

NameError: ignored

In [86]:
train_generator.filenames

[]

In [76]:
train_generator.

In [64]:

# Preprocess the labels using the get_class_label function

#train_generator.labels = labels  # Set the labels for the generator
#train_generator.classes = labels
#print(folder_path_unlabeled,labels)

[]


In [28]:
# Create a pipeline

# Data Preparation
# Resize image
for image in os.listdir(folder_path_unlabeled):
  try:
    tnt_image = cv2.imread(folder_path_unlabeled+image)
    tnt_image = cv2.resize(tnt_image, (224,224))

    # Normalize the pixel values to the range [0, 1]
    image_array = np.array(tnt_image)
    n_image_array = image_array/255





  except TypeError as e:
    print(e)

Streaming output truncated to the last 5000 lines.
  [78 78 78]
  [77 77 77]]]
[[[ 76  76  76]
  [ 75  75  75]
  [ 75  75  75]
  ...
  [ 77  77  77]
  [ 77  77  77]
  [ 76  76  76]]

 [[ 75  75  75]
  [ 75  75  75]
  [ 73  73  73]
  ...
  [ 80  80  80]
  [ 77  77  77]
  [ 76  76  76]]

 [[ 74  74  74]
  [ 75  75  75]
  [ 75  75  75]
  ...
  [ 82  82  82]
  [ 79  79  79]
  [ 76  76  76]]

 ...

 [[ 77  77  77]
  [ 77  77  77]
  [ 78  78  78]
  ...
  [108 108 108]
  [101 101 101]
  [ 97  97  97]]

 [[ 76  76  76]
  [ 78  78  78]
  [ 77  77  77]
  ...
  [ 97  97  97]
  [ 97  97  97]
  [ 92  92  92]]

 [[ 77  77  77]
  [ 78  78  78]
  [ 77  77  77]
  ...
  [ 92  92  92]
  [ 92  92  92]
  [ 92  92  92]]]
[[[77 77 77]
  [76 76 76]
  [78 78 78]
  ...
  [86 86 86]
  [84 84 84]
  [84 84 84]]

 [[75 75 75]
  [74 74 74]
  [77 77 77]
  ...
  [84 84 84]
  [81 81 81]
  [82 82 82]]

 [[75 75 75]
  [77 77 77]
  [77 77 77]
  ...
  [81 81 81]
  [83 83 83]
  [82 82 82]]

 ...

 [[93 93 93]
  [92 92 92]
 